# fuzzy_pandas examples

These are almost all from [Max Harlow](https://twitter.com/maxharlow)'s [awesome NICAR2019 presentation](https://docs.google.com/presentation/d/1djKgqFbkYDM8fdczFhnEJLwapzmt4RLuEjXkJZpKves/) where he demonstrated [csvmatch](https://github.com/maxharlow/csvmatch), which fuzzy_pandas is based on.

**SCROLL DOWN DOWN DOWN TO GET TO THE FUZZY MATCHING PARTS.**

In [1]:
import pandas as pd
import fuzzy_pandas as fpd

In [2]:
df1 = pd.read_csv("data/data1.csv")
df2 = pd.read_csv("data/data2.csv")

In [3]:
df1

,name,location,codename
0,George Smiley,London,Beggerman
1,Percy Alleline,London,Tinker
2,Roy Bland,London,Soldier
3,Toby Esterhase,Vienna,Poorman
4,Peter Guillam,Brixton,none
5,Bill Haydon,London,Tailor
6,Oliver Lacon,London,none
7,Jim Prideaux,Slovakia,none
8,Connie Sachs,Oxford,none


In [4]:
df2

,Person Name,Location
0,Maria Andreyevna Ostrakova,Russia
1,Otto Leipzig,Estonia
2,George SMILEY,London
3,Peter Guillam,Brixton
4,Konny Saks,Oxford
5,Saul Enderby,London
6,Sam Collins,Vietnam
7,Tony Esterhase,Vienna
8,Claus Kretzschmar,Hamburg


# Exact matches

By default, all columns from both dataframes are returned.

In [5]:
# csvmatch \
#     forbes-billionaires.csv \
#     bloomberg-billionaires.csv \
#     --fields1 name \
#     --fields2 Name

df1 = pd.read_csv("data/forbes-billionaires.csv")
df2 = pd.read_csv("data/bloomberg-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Name')

print("Found", results.shape)
results.head(5)

Found (354, 19)


,name,lastName,uri,imageUri,worthChange,source,industry,gender,country,timestamp,realTimeWorth,realTimeRank,realTimePosition,squareImage,Rank,Name,Total_net_worth,Country,Industry
0,Alexander Otto,Otto,alexander-otto,no-pic,2.120,real estate,Real Estate,M,Germany,1547575201867,10821.927,126.0,126.0,//specials-images.forbesimg.com/imageserve/5a7...,323,Alexander Otto,$4.94B,Germany,Real Estate
1,Ben Ashkenazy,Ashkenazy,ben-ashkenazy,no-pic,0.000,real estate,Real Estate,M,United States,1547574901333,4000.000,499.0,499.0,//specials-images.forbesimg.com/imageserve/59e...,447,Ben Ashkenazy,$4.05B,United States,Real Estate
2,Giovanni Ferrero,Ferrero,giovanni-ferrero,no-pic,0.000,"Nutella, chocolates",Food and Beverage,M,Italy,1547575201866,22673.165,38.0,38.0,//specials-images.forbesimg.com/imageserve/5b1...,33,Giovanni Ferrero,$22.6B,Italy,Food & Beverage
3,Henry Cheng,Cheng,henry-cheng-1,no-pic,3.542,property,Diversified,M,Hong Kong,1547574901334,1334.282,1630.0,1630.0,//specials-images.forbesimg.com/imageserve/5a7...,79,Henry Cheng,$14.1B,Hong Kong,Retail
4,Henry Laufer,Laufer,henry-laufer,no-pic,0.000,hedge funds,Finance and Investments,M,United States,1547574901333,2000.000,1141.0,1142.0,NaN,463,Henry Laufer,$3.95B,United States,Finance


### Only keeping matching columns

The csvmatch default only gives you the shared columns, which you can reproduce with `keep='match'`

In [6]:
df1 = pd.read_csv("data/forbes-billionaires.csv")
df2 = pd.read_csv("data/bloomberg-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2, left_on='name', right_on='Name', keep='match')

print("Found", results.shape)
results.head(5)

Found (354, 2)


,name,Name
0,Alexander Otto,Alexander Otto
1,Ben Ashkenazy,Ben Ashkenazy
2,Giovanni Ferrero,Giovanni Ferrero
3,Henry Cheng,Henry Cheng
4,Henry Laufer,Henry Laufer


### Only keeping specified columns

In [7]:
df1 = pd.read_csv("data/forbes-billionaires.csv")
df2 = pd.read_csv("data/bloomberg-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='Name',
            keep_left=['name', 'realTimeRank'],
            keep_right=['Rank'])

print("Found", results.shape)
results.head(5)

Found (354, 3)


,name,realTimeRank,Rank
0,Alexander Otto,126.0,323
1,Ben Ashkenazy,499.0,447
2,Giovanni Ferrero,38.0,33
3,Henry Cheng,1630.0,79
4,Henry Laufer,1141.0,463


## Case sensitivity

This one doesn't give us any results!

In [8]:
# csvmatch \
#     cia-world-leaders.csv \
#     davos-attendees-2019.csv \
#     --fields1 name \
#     --fields2 full_name

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/davos-attendees-2019.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='full_name',
            keep='match')

print("Found", results.shape)
results.head(10)

Found (0, 2)


,name,full_name


But if we add **ignore_case** we are good to go.

In [9]:
# csvmatch \
#     cia-world-leaders.csv \
#     davos-attendees-2019.csv \
#     --fields1 name \
#     --fields2 full_name \
#     --ignore-case \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/davos-attendees-2019.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='full_name',
            ignore_case=True,
            keep='match')

print("Found", results.shape)
results.head(5)

Found (119, 2)


,name,full_name
0,Abdelkader MESSAHEL,Abdelkader Messahel
1,Dante SICA,Dante Sica
2,Guido SANDLERIS,Guido Sandleris
3,Nikol PASHINYAN,Nikol Pashinyan
4,Steven CIOBO,Steven Ciobo


### Ignoring case, non-latin characters, word ordering

You should really be reading [the presentation](https://docs.google.com/presentation/d/1djKgqFbkYDM8fdczFhnEJLwapzmt4RLuEjXkJZpKves/edit)!

In [10]:
# $ csvmatch \
# cia-world-leaders.csv \
# davos-attendees-2019.csv \
# --fields1 name \
# --fields2 full_name \
# -i -a -n -s \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/davos-attendees-2019.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on=['name'],
            right_on=['full_name'],
            ignore_case=True,
            ignore_nonalpha=True,
            ignore_nonlatin=True,
            ignore_order_words=True,
            keep='match')

print("Found", results.shape)
results.head(5)

Found (138, 2)


,name,full_name
0,Abdelkader MESSAHEL,Abdelkader Messahel
1,Dante SICA,Dante Sica
2,Nicolas DUJOVNE,Nicolás Dujovne
3,Guido SANDLERIS,Guido Sandleris
4,Nikol PASHINYAN,Nikol Pashinyan


# Fuzzy matching

## Levenshtein: Edit distance

In [11]:
# csvmatch \
#     cia-world-leaders.csv \
#     forbes-billionaires.csv \
#     --fields1 name \
#     --fields2 name \
#     --fuzzy levenshtein \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/forbes-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='name',
            method='levenshtein',
            keep='match')

print("Found", results.shape)
results.head(10)

Found (323, 2)


,name,name
0,Mohammad Ibrahim SHINWARI,Mohammed Ibrahim
1,Mohamed AISSA,Mohammed Al Issa
2,Mohamed MEBARKI,Mohamed Mansour
3,Michael BROWNE,Michael Otto
4,Michael BROWNE,Michael Rubin
5,Christian PORTER,Christian Oetker
6,Christopher PYNE,J. Christopher Reyes
7,Christopher PYNE,Christopher Hohn
8,Christopher PYNE,Christopher Cline
9,Michael KEENAN,Michael Kim


### Setting a threshold with Levenshtein

In [12]:
# csvmatch \
#     cia-world-leaders.csv \
#     forbes-billionaires.csv \
#     --fields1 name \
#     --fields2 name \
#     --fuzzy levenshtein \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/forbes-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='name',
            method='levenshtein',
            threshold=0.85,
            keep='match')

print("Found", results.shape)
results.head(10)

Found (0, 2)


,name,name


## Jaro: Edit distance

In [13]:
# csvmatch \
#     cia-world-leaders.csv \
#     forbes-billionaires.csv \
#     --fields1 name \
#     --fields2 name \
#     --fuzzy levenshtein \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/forbes-billionaires.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='name',
            method='jaro',
            keep='match')

print("Found", results.shape)
results.head(10)

Found (77842, 2)


,name,name
0,Hamdullah MOHIB,Pamela Mars
1,Hamdullah MOHIB,Harold Hamm
2,Hamdullah MOHIB,Samuel Yin
3,Hamdullah MOHIB,Kalanithi Maran
4,Hamdullah MOHIB,Saleh Kamel
5,Hamdullah MOHIB,Hamdi Ulukaya
6,Hamdullah MOHIB,Emanuella Barilla
7,Hamdullah MOHIB,Hamdi Akin
8,Mohammad Farid HAMIDI,Fahd Hariri
9,Mohammad Farid HAMIDI,Mikhail Fridman


## Metaphone: Phonetic match

In [14]:
# csvmatch \
#     cia-world-leaders.csv \
#     un-sanctions.csv \
#     --fields1 name \
#     --fields2 name \
#     --fuzzy metaphone \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/un-sanctions.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='name',
            method='metaphone',
            keep='match')

print("Found", results.shape)
results.head(10)

Found (18, 2)


,name,name
0,KYAW TIN,KO TAE HUN
1,CHHAY THAN,KO TAE HUN
2,MIAO Wei,M23
3,Aysha MOHAMMED,JAISH-I-MOHAMMED
4,Mamadi TOURE,MAMADU TURE
5,KIM Yong Chol,KIM YONG CHOL
6,PAK Yong Sik,PAK YONG SIK
7,RI Man Gon,RI MAN GON
8,RI Su Yong,RI SU YONG
9,JON Kyong Nam,YO'N CHO'NG NAM


## Bilenko

You'll need to respond to the prompts when you run the code. 10-15 is best, but send `f` when you've decided you're finished.

In [15]:
# $ csvmatch \
# cia-world-leaders.csv \
# davos-attendees-2019.csv \
# --fields1 name \
# --fields2 full_name \
# --fuzzy bilenko \

df1 = pd.read_csv("data/cia-world-leaders.csv")
df2 = pd.read_csv("data/davos-attendees-2019.csv")

results = fpd.fuzzy_merge(df1, df2,
            left_on='name',
            right_on='full_name',
            method='bilenko',
            keep='match')

print("Found", results.shape)
results.head(10)


Answer questions as follows:
 y - yes
 n - no
 s - skip
 f - finished

name: Antonio Eduardo BECALI Garrido

name: Antonio Neri

Do these records refer to the same thing? [y/n/s/f] 

n



name: Michel Bongongo IKOLI

name: Michel Vounatsos

Do these records refer to the same thing? [y/n/s/f] 

n



name: Mariam Mahamat NOUR

name: Mariam Mohammed Saeed Hareb Al Mehairi

Do these records refer to the same thing? [y/n/s/f] 

...


name: Joaquim LIMA

name: Joachim Wenning

Do these records refer to the same thing? [y/n/s/f] 

f


INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (firstTokenPredicate, name), SimplePredicate: (sameSevenCharStartPredicate, name))
INFO:rlr.crossvalidation:using cross validation to find optimum alpha...
/Users/soma/.pyenv/versions/3.6.8/lib/python3.6/site-packages/rlr/crossvalidation.py:122: RuntimeWarning: invalid value encountered in double_scalars
  * (true_distinct + false_distinct)))
INFO:rlr.crossvalidation:optimum alpha: 0.000010, score 0.1
INFO:dedupe.training:Final predicate set:
INFO:dedupe.training:(SimplePredicate: (firstTokenPredicate, name), SimplePredicate: (sameSevenCharStartPredicate, name))
INFO:dedupe.api:0 records
INFO:dedupe.api:100 records
INFO:dedupe.api:200 records
INFO:dedupe.api:300 records
INFO:dedupe.api:400 records
INFO:dedupe.api:500 records
INFO:dedupe.api:600 records
INFO:dedupe.api:700 records
INFO:dedupe.api:800 records
INFO:dedupe.api:900 records
INFO:dedupe.api:1000 records
INFO:dedupe.api:1100 records
INFO:dedupe.ap

Found (241, 2)


,name,full_name
0,Raj Kumar SINGH,Raj Kumar
1,Tran Hong HA,Tran Hong Ha
2,Antonio Serifo EMBALO,Antonio Neri
3,Charles Richard MONDJO,Charles Li
4,"Anthony Wayne Jerome PHILLIPS-SPENCER, Brig. G...",Anthony Tan
5,David Cruiser NGCAMPHALALA,David Craig
6,Michael Daniel HIGGINS,Michael Ding
7,Michael Tiangjiek MUT,Michael Ryan
8,Tran Tuan ANH,Tran Tuan Anh
9,James Gordon CARR,James Gorman
